In [1]:
from config import *
from utilities import get_model_name
from dataset import ClassificationDataset

import functools
import numpy as np
import pandas as pd

results_dict = {}
internal, external = 11, 12
encode_method = "dummy"
impute_method = "itbr"
fs_method, fs_ratio = "chi2", 0.5
norm_method = "maxmin"
classification_dataset = ClassificationDataset(
    internal, external, encode_method, impute_method, fs_method, fs_ratio, norm_method, random_state=SEED)

In [2]:
# from sklearn.neural_network import MLPClassifier
from utilities import MLPClassifier

model_fn = functools.partial(
    MLPClassifier, random_state=SEED, max_iter=50,
    momentum=0.9, squares=0.999, optimizer_t='sgdm',
    use_residual=True, use_batch_norm=False)
param_grid_list = [{
    # hidden_layer_sizes = (64, 32)
    # config       lr_range            worst_auc
    # adam w/ bn:  [2e-5, 5e-5, 1e-4]  0.8942
    # adam w/o bn: [5e-5, 1e-4, 2e-4]  0.9148
    # sgdm w/ bn:  [1e-3, 2e-3, 5e-3]  0.8982
    # sgdm w/o bn: [5e-3, 1e-2, 2e-2]  0.9177

    'hidden_layer_sizes': [(128, 128), (64, 64, 64)],
    'learning_rate': [5e-3, 1e-2, 2e-2],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

MLPClassifier


model(hidden_layer_sizes=(128, 128), learning_rate=0.005)
Valid:         AUC: 0.9105 | AUPR: 0.6311 | Acc: 95.15%


model(hidden_layer_sizes=(128, 128), learning_rate=0.01)
Valid:         AUC: 0.9144 | AUPR: 0.6456 | Acc: 95.26%


model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9164 | AUPR: 0.6532 | Acc: 95.36%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.005)
Valid:         AUC: 0.9108 | AUPR: 0.6325 | Acc: 95.15%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.01)
Valid:         AUC: 0.9148 | AUPR: 0.6466 | Acc: 95.29%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9160 | AUPR: 0.6518 | Acc: 95.27%
best model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9164 | AUPR: 0.6532 | Acc: 95.36%
Internal test: AUC: 0.9149 | AUPR: 0.6486 | Acc: 94.70%
External test: AUC: 0.9152 | AUPR: 0.6395 | Acc: 94.82%


In [3]:
from sklearn.svm import SVC

model_fn = functools.partial(
    SVC, random_state=SEED, probability=True,
    class_weight='balanced')
param_grid_list = [{
    'kernel': ['linear'],
    'C': [0.01, 0.1, 1],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

SVC
model(kernel=linear, C=0.01)
Valid:         AUC: 0.9121 | AUPR: 0.6148 | Acc: 94.89%
model(kernel=linear, C=0.1)
Valid:         AUC: 0.9134 | AUPR: 0.6162 | Acc: 94.91%
model(kernel=linear, C=1)
Valid:         AUC: 0.9133 | AUPR: 0.6148 | Acc: 94.88%
best model(kernel=linear, C=0.1)
Valid:         AUC: 0.9134 | AUPR: 0.6162 | Acc: 94.91%
Internal test: AUC: 0.9120 | AUPR: 0.6351 | Acc: 94.67%
External test: AUC: 0.9099 | AUPR: 0.5849 | Acc: 94.30%


In [4]:
from sklearn.neighbors import KNeighborsClassifier

model_fn = functools.partial(
    KNeighborsClassifier, n_jobs=-1, algorithm='auto', p=2)
param_grid_list = [{
    'n_neighbors': [3, 5, 7],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

KNeighborsClassifier
model(n_neighbors=3)
Valid:         AUC: 0.7400 | AUPR: 0.4064 | Acc: 94.48%
model(n_neighbors=5)
Valid:         AUC: 0.7689 | AUPR: 0.4543 | Acc: 94.92%
model(n_neighbors=7)
Valid:         AUC: 0.7861 | AUPR: 0.4783 | Acc: 94.95%
best model(n_neighbors=7)
Valid:         AUC: 0.7861 | AUPR: 0.4783 | Acc: 94.95%
Internal test: AUC: 0.7907 | AUPR: 0.4769 | Acc: 94.50%
External test: AUC: 0.7959 | AUPR: 0.4752 | Acc: 94.76%


In [5]:
from sklearn.naive_bayes import GaussianNB

model_fn = GaussianNB
param_grid_list = [{}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GaussianNB
model()
Valid:         AUC: 0.8583 | AUPR: 0.3688 | Acc: 84.01%
best model()
Valid:         AUC: 0.8583 | AUPR: 0.3688 | Acc: 84.01%
Internal test: AUC: 0.8665 | AUPR: 0.3875 | Acc: 84.18%
External test: AUC: 0.8513 | AUPR: 0.3425 | Acc: 82.65%


In [6]:
# from sklearn.exceptions import ConvergenceWarning
# import warnings
# warnings.filterwarnings("ignore", category=ConvergenceWarning)
from sklearn.linear_model import LogisticRegression

model_fn = functools.partial(
    LogisticRegression, random_state=SEED, n_jobs=-1, 
    max_iter=2000, solver='saga', penalty='l2',
    )
param_grid_list = [{
    'C': [1.0, 10.0, 100.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(
    model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(
    best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

LogisticRegression
model(C=1.0)
Valid:         AUC: 0.9138 | AUPR: 0.6366 | Acc: 95.16%
model(C=10.0)
Valid:         AUC: 0.9136 | AUPR: 0.6353 | Acc: 95.15%
model(C=100.0)
Valid:         AUC: 0.9135 | AUPR: 0.6351 | Acc: 95.15%
best model(C=1.0)
Valid:         AUC: 0.9138 | AUPR: 0.6366 | Acc: 95.16%
Internal test: AUC: 0.9146 | AUPR: 0.6522 | Acc: 94.79%
External test: AUC: 0.9092 | AUPR: 0.6090 | Acc: 94.59%


In [7]:
from sklearn.tree import DecisionTreeClassifier

model_fn = functools.partial(
    DecisionTreeClassifier, random_state=SEED,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

DecisionTreeClassifier
model(criterion=gini)
Valid:         AUC: 0.8730 | AUPR: 0.5383 | Acc: 84.12%
model(criterion=log_loss)
Valid:         AUC: 0.8768 | AUPR: 0.5623 | Acc: 85.43%
model(criterion=entropy)
Valid:         AUC: 0.8768 | AUPR: 0.5623 | Acc: 85.43%
best model(criterion=log_loss)
Valid:         AUC: 0.8768 | AUPR: 0.5623 | Acc: 85.43%
Internal test: AUC: 0.8950 | AUPR: 0.6167 | Acc: 84.87%
External test: AUC: 0.8837 | AUPR: 0.5803 | Acc: 84.57%


In [8]:
from sklearn.ensemble import RandomForestClassifier

model_fn = functools.partial(
    RandomForestClassifier, random_state=SEED, n_jobs=-1,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5, bootstrap=False)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
        'n_estimators': [50, 100, 200],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

RandomForestClassifier
model(criterion=gini, n_estimators=50)
Valid:         AUC: 0.9104 | AUPR: 0.6351 | Acc: 87.75%
model(criterion=gini, n_estimators=100)
Valid:         AUC: 0.9117 | AUPR: 0.6367 | Acc: 88.03%
model(criterion=gini, n_estimators=200)
Valid:         AUC: 0.9122 | AUPR: 0.6370 | Acc: 88.02%
model(criterion=log_loss, n_estimators=50)
Valid:         AUC: 0.9098 | AUPR: 0.6362 | Acc: 88.00%
model(criterion=log_loss, n_estimators=100)
Valid:         AUC: 0.9117 | AUPR: 0.6394 | Acc: 88.19%
model(criterion=log_loss, n_estimators=200)
Valid:         AUC: 0.9121 | AUPR: 0.6396 | Acc: 88.16%
model(criterion=entropy, n_estimators=50)
Valid:         AUC: 0.9098 | AUPR: 0.6362 | Acc: 88.00%
model(criterion=entropy, n_estimators=100)
Valid:         AUC: 0.9117 | AUPR: 0.6394 | Acc: 88.19%
model(criterion=entropy, n_estimators=200)
Valid:         AUC: 0.9121 | AUPR: 0.6396 | Acc: 88.16%
best model(criterion=gini, n_estimators=200)
Valid:         AUC: 0.9122 | AUPR: 0.6370 | Acc: 8

In [9]:
from sklearn.ensemble import GradientBoostingClassifier

model = functools.partial(
    GradientBoostingClassifier, random_state=SEED,
    learning_rate=0.1, max_depth=5, loss='log_loss', 
    n_estimators=100)
param_grid_list = [{
        'criterion': ['friedman_mse', 'squared_error'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GradientBoostingClassifier
model(criterion=friedman_mse)
Valid:         AUC: 0.9180 | AUPR: 0.6432 | Acc: 95.11%
model(criterion=squared_error)
Valid:         AUC: 0.9184 | AUPR: 0.6484 | Acc: 95.15%
best model(criterion=squared_error)
Valid:         AUC: 0.9184 | AUPR: 0.6484 | Acc: 95.15%
Internal test: AUC: 0.9168 | AUPR: 0.6401 | Acc: 94.84%
External test: AUC: 0.9121 | AUPR: 0.6180 | Acc: 94.83%


In [10]:
from sklearn.ensemble import AdaBoostClassifier
model = functools.partial(AdaBoostClassifier, random_state=SEED, algorithm="SAMME")
param_grid_list = [{
    'n_estimators': [100, 200, 500],
    'learning_rate': [1.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

AdaBoostClassifier
model(n_estimators=100, learning_rate=1.0)
Valid:         AUC: 0.9098 | AUPR: 0.6226 | Acc: 94.98%
model(n_estimators=200, learning_rate=1.0)
Valid:         AUC: 0.9125 | AUPR: 0.6273 | Acc: 94.96%
model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9147 | AUPR: 0.6315 | Acc: 94.99%
best model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9147 | AUPR: 0.6315 | Acc: 94.99%
Internal test: AUC: 0.9095 | AUPR: 0.6430 | Acc: 94.65%
External test: AUC: 0.9073 | AUPR: 0.5987 | Acc: 94.44%


In [11]:
dataframe = pd.DataFrame(results_dict).transpose().reset_index()
dataframe.columns = ['model', 'val_auc', 'val_aupr', 'val_acc', 'intest_auc', 'intest_aupr', 'intest_acc', 'extest_auc', 'extest_aupr', 'extest_acc']
# dataframe.to_csv()
filename = os.path.join(RESULTS_DIR, encode_method, impute_method, f'{fs_method}{int(fs_ratio * 142)}.csv')
dirname = os.path.dirname(filename)
if not os.path.exists(dirname):
    os.makedirs(dirname)
dataframe.to_csv(filename, index=False)